## Optimization Modelling in Python

https://medium.com/analytics-vidhya/optimization-modelling-in-python-scipy-pulp-and-pyomo-d392376109f4

Let’s consider simplified transportation type problem. <br>
We have set of customers $I = [1,2,3,4,5]$ and set of factories $J = [1,2,3]$. <br>
Each customer has some fixed product demand $d = [80, 270, 250, 160, 180]$ and each factory has fixed production capacity $M = [500, 500, 500]$.<br>
There are also fixed transportation costs $c_{ij}$ to deliver one unit of good from factory $j$ to customer $i$.

\begin{equation}
\begin{split}
\text{minimize} \quad & \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij} \\
\text{subject to} \quad & \sum_{j \in J} x_{ij} = d_{ij}, \forall i \in I  \\      
                & \sum_{i \in I} x_{ij} \leq M_j, \forall j \in J \\
                & x_{ij} \geq 0, \forall i \in I, j \in J
\end{split}
\end{equation} 

* objective function ($\text{min total cost}$) — find such values of decision variables that total transportation cost is the lowest (linear expression in this case)
* decision variables $x_{ij}$ — quantities of goods to be sent from factory $j$ to customer $i$ (positive real numbers)
* constraints — total amount of goods must satisfy both customer demand and factory production capacity (equalities/inequalities that have linear expression on the left-hand side

In [1]:
import sys
import numpy as np
d = {1:80, 2:270, 3:250, 4:160, 5:180}  # customer demand
M = {1:500, 2:500, 3:500}               # factory capacity
I = [1,2,3,4,5]                         # Customers
J = [1,2,3]                             # Factories
cost = {(1,1):4,    (1,2):6,    (1,3):9,
     (2,1):5,    (2,2):4,    (2,3):7,
     (3,1):6,    (3,2):3,    (3,3):3,
     (4,1):8,    (4,2):5,    (4,3):3,
     (5,1):10,   (5,2):8,    (5,3):4
   }                                    # transportation costs

In [2]:
# pip install pyomo
from pyomo import environ as pyo

In [3]:
# ConcreteModel is model where data values supplied at the time of the model definition. As opposite to AbstractModel where data values are supplied in data file
model = pyo.ConcreteModel()

# all iterables are to be converted into Set objects
model.d_cust_demand = pyo.Set(initialize = d.keys())
model.M_fact_capacity = pyo.Set(initialize = M.keys())

Basic elements of a linear programming model with Pyomo:
* Variables: They represent the decisions to be made in the optimization problem. <br>
In Pyomo, they are defined using the Var class.<br>
They can be of different types: continuous (NonNegativeReals), integers (Integer), binary (Binary), etc. <br>

In [4]:
# Parameters
# Cartesian product of two sets creates list of tuples [((i1,j1),v1),((i2,j2),v2),...] !!!
model.transport_cost = pyo.Param(
    model.d_cust_demand * model.M_fact_capacity,
    initialize = cost,
    within = pyo.NonNegativeReals)

model.cust_demand = pyo.Param(model.d_cust_demand, 
    initialize = d,
    within = pyo.NonNegativeReals)

model.fact_capacity = pyo.Param(model.M_fact_capacity, 
    initialize = M,
    within = pyo.NonNegativeReals)

$\text{subject to} \quad x_{ij} \geq 0, \forall i \in I, j \in J$

* Constraints: Restrictions that must be met by variables. <br>
In Pyomo, they are defined using the Constraint class. <br>
They are expressed as equations or inequalities. <br>

In [5]:
model.x = pyo.Var(
    model.d_cust_demand * model.M_fact_capacity,
    domain = pyo.NonNegativeReals,
    bounds = (0, max(d.values())))

$\text{minimize} \quad \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij} $

* Objective: Function to be optimized (maximized or minimized).<br>
In Pyomo, it is defined using the Objective function. <br>
It must be a linear expression <br>

In [6]:
model.objective = pyo.Objective(
    expr = pyo.summation(model.transport_cost, model.x),
    sense = pyo.minimize)

$\text{subject to} \quad \sum_{j \in J} x_{ij} = d_{ij}, \forall i \in I  $

In [7]:
# Constraints: sum of goods == customer demand
def meet_demand(model, customer):
    sum_of_goods_from_factories = sum(model.x[customer,factory] for factory in model.M_fact_capacity)
    customer_demand = model.cust_demand[customer]
    return sum_of_goods_from_factories == customer_demand
    
model.Constraint1 = pyo.Constraint(model.d_cust_demand, rule = meet_demand)

$\text{subject to} \quad \sum_{i \in I} x_{ij} \leq M_j, \forall j \in J $

In [8]:
# Constraints: sum of goods <= factory capacity
def meet_capacity(model, factory):
    sum_of_goods_for_customers = sum(model.x[customer,factory] for customer in model.d_cust_demand)
    factory_capacity = model.fact_capacity[factory]
    return sum_of_goods_for_customers <= factory_capacity
    
model.Constraint2 = pyo.Constraint(model.M_fact_capacity, rule = meet_demand)

In [9]:
# conda install glpk
solver = pyo.SolverFactory("glpk")
solution = solver.solve(model)

In [10]:
from pyomo.opt import SolverStatus, TerminationCondition
if (solution.solver.status == SolverStatus.ok) and (solution.solver.termination_condition == TerminationCondition.optimal):
    print("Solution is feasible and optimal")
    print("Objective function value = ", model.objective())
elif solution.solver.termination_condition == TerminationCondition.infeasible:
    print ("Failed to find solution.")
else:
    # something else is wrong
    print(str(solution.solver))
assignments = model.x.get_values().items()
EPS = 1.e-6
for (customer,factory),x in sorted(assignments):
    if x > EPS:
        print("sending quantity %10s from factory %3s to customer %3s" % (x, factory, customer))

Solution is feasible and optimal
Objective function value =  3350.0
sending quantity       80.0 from factory   1 to customer   1
sending quantity      270.0 from factory   2 to customer   2
sending quantity      250.0 from factory   3 to customer   3
sending quantity      160.0 from factory   3 to customer   4
sending quantity      180.0 from factory   3 to customer   5
